In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import time

In [5]:
# Read into CSV files
df = pd.read_csv('data/uk_data.csv', encoding='unicode_escape')

In [7]:
print(df.shape)
df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [8]:
# Get data for product/customer matrix
data = df[['CustomerID', 'StockCode', 'Quantity']]
print(data.shape)
data.head()

(541909, 3)


,CustomerID,StockCode,Quantity
0,17850.0,85123A,6
1,17850.0,71053,6
2,17850.0,84406B,8
3,17850.0,84029G,6
4,17850.0,84029E,6


In [10]:
# Group by to get purchase quantity by customer ID and stock code
data = data.groupby(['CustomerID','StockCode']) \
    .agg({'Quantity':'sum'}) \
    .rename(columns={'StockCode':'ProductID'}) \
    .reset_index()
print(data.shape)
data.head()

(267615, 3)


,CustomerID,StockCode,Quantity
0,12346.0,23166,0
1,12347.0,16008,24
2,12347.0,17021,36
3,12347.0,20665,6
4,12347.0,20719,40


In [15]:
# Question - get rid of negative values?
data[data['CustomerID']==17850]

,CustomerID,StockCode,Quantity
251481,17850.0,15056BL,24
251482,17850.0,20679,42
251483,17850.0,21068,96
251484,17850.0,21071,90
251485,17850.0,21169,-2
251486,17850.0,21730,102
251487,17850.0,21871,83
251488,17850.0,21874,-1
251489,17850.0,22411,60
251490,17850.0,22632,102


In [16]:
# Create dummy - park this!
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)
data_dummy

,CustomerID,StockCode,Quantity,purchase_dummy
0,12346.0,23166,0,1
1,12347.0,16008,24,1
2,12347.0,17021,36,1
3,12347.0,20665,6,1
4,12347.0,20719,40,1
...,...,...,...,...
267610,18287.0,84920,4,1
267611,18287.0,85039A,96,1
267612,18287.0,85039B,120,1
267613,18287.0,85040A,48,1


In [18]:
# Function to normalize item values across users
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='Quantity', index='CustomerID', columns='StockCode')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()

In [19]:
normalize_data(data)

MemoryError: Unable to allocate 246. MiB for an array with shape (2, 16106448) and data type float64